In [ ]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt
from numpy.random import *
import torch

In [ ]:
import torch, random

size = 128
Time = 10000
stepLoop = 1000
dx = 0.5
dt = 0.5
#Diffusion Coefficients
du = 0.1
dv = 1.0
loopNumber = int(Time/dt/stepLoop)


if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

def Torch_Turing(fu,fv,gu,gv,q):
    u1 = torch.rand(size,size)-0.5
    v1 = torch.rand(size,size)-0.5
    u2 = torch.zeros((size,size))
    v2 = torch.zeros((size,size))


    Ku = torch.zeros((size,size))
    Kv = torch.zeros((size,size))

    for x in [u1,v1,u2,v2,Ku,Kv]:
        x.requires_grad = False
        x = x.to(device)

    Ku[0,0]=1+4*du/dx/dx*dt
    Ku[1,0]=-du/dx/dx*dt
    Ku[-1,0]=-du/dx/dx*dt
    Ku[0,1]=-du/dx/dx*dt
    Ku[0,-1]=-du/dx/dx*dt

    Kv[0,0]=1+4*dv/dx/dx*dt
    Kv[1,0]=-dv/dx/dx*dt
    Kv[-1,0]=-dv/dx/dx*dt
    Kv[0,1]=-dv/dx/dx*dt
    Kv[0,-1]=-dv/dx/dx*dt

    Kufi=1/torch.fft.fft2(Ku)
    Kvfi=1/torch.fft.fft2(Kv)

    def func_u(x,y,Q):
      return fu*x - fv*y - Q*x**2 - x**3

    def func_v(x,y):
      return gu*x - gv*y

    diff_list = np.zeros(2)
    for t in range(loopNumber):
        for i in range(stepLoop):
            u2 = torch.fft.ifft2(torch.fft.fft2(
                u1 + dt*func_u(u1,v1,q)
                )*Kufi)
            v2 = torch.fft.ifft2(torch.fft.fft2(
                v1 + dt*func_v(u1,v1)
                )*Kvfi)
            u1 = torch.fft.ifft2(torch.fft.fft2(
                u2 + dt*func_u(u2,v2,q)
                )*Kufi)
            v1 = torch.fft.ifft2(torch.fft.fft2(
                v2 + dt*func_v(u2,v2)
                )*Kvfi)


        #Failed to generate a pattern
        if torch.var(u1.reshape(size*size))<0.0005:
          return False

    end = time.time()
    return u1.detach().cpu()

def post_process(u,display=True):
    h = u.numpy()
    h = h.astype(float)
    h = h-h.min()
    h = h/h.max()*255
    h = h.astype(np.uint8)
    if display:
        plt.imshow(h)
        plt.show()
    return h

In [ ]:
result = Torch_Turing(fu,fv,gu,gv,q)
image = post_process(result,display=True)
plt.imshow(image)
plt.show()